Na lemmatization também vamos buscar reduzir a palavra à sua raiz, retirando todas as inflexões e chegando ao lemma. 
Porém, essa redução sempre resultará em uma palavra que realmente existe na gramática. Outro ponto importante é que, 
nessa técnica, a classe gramatical da palavra será levada em consideração para fazer a redução. 
Vamos agora utilizar a biblioteca spaCy para aplicar essa técnica.

In [22]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Lucas
[nltk_data]     Araújo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
bossa = pd.read_csv('datasets/bossa_nova_songs_dataset.csv')
samba = pd.read_csv('datasets/samba_dataset.csv',sep='|')
axe = pd.read_csv('datasets/letras_mus_br_axe.csv')
funk = pd.read_csv('datasets/Dataset_funk_carioca.csv')
gospel = pd.read_csv('datasets/gospel.csv')

In [24]:
gospel_ajustado = pd.DataFrame(gospel['letra'])  #OK 55000
gospel_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
gospel_ajustado.drop(gospel_ajustado.tail(51498).index,inplace = True) #3000
samba_ajustado = pd.DataFrame(samba['letra'])  #OK 3000
samba_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
samba_ajustado.drop(samba_ajustado.tail(118).index,inplace = True) #3000
axe_ajustado = pd.DataFrame(axe['letras'])  #OK1000
axe_ajustado.rename(columns={'letras': 'lyric'}, inplace = True) #OK
funk_ajustado = pd.DataFrame(funk['letra'])  #OK 1000
funk_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
bossa_ajustado = pd.DataFrame(bossa['song_lyrics'])  #OK 6000
bossa_ajustado.rename(columns={'song_lyrics': 'lyric'}, inplace = True) #OK
bossa_ajustado.drop(bossa_ajustado.tail(3106).index,inplace = True) #3000

In [25]:
bossa_ajustado['gen'] = 'Bossa Nova'
funk_ajustado['gen'] = 'Funk'
gospel_ajustado['gen'] = 'Gospel'
axe_ajustado['gen'] = 'Axe'
samba_ajustado['gen'] = 'Samba'

In [26]:
datasets = [bossa_ajustado,funk_ajustado,bossa_ajustado,axe_ajustado,samba_ajustado]

In [27]:
lyrics = pd.concat(datasets)
lyrics.reset_index(drop=True,inplace=True)
lyrics

,lyric,gen
0,"De tudo, ao meu amor serei atento antes E co...",Bossa Nova
1,Era uma casa Muito engraçada Não tinha teto ...,Bossa Nova
2,E por falar em saudade Onde anda você Onde a...,Bossa Nova
3,Filhos... Filhos? Melhor não tê-los! Mas se ...,Bossa Nova
4,É melhor ser alegre que ser triste Alegria é...,Bossa Nova
...,...,...
10995,( Não tenho lar Não tenho amor Pra que vou me ...,Samba
10996,o Senhor é a minha forçao meu escudo...minha ...,Samba
10997,Sorte é sorte o malandro também tem seu dia d...,Samba
10998,"A gente briga sem querer Chora ""prá"" voltar F...",Samba


In [28]:
import spacy

def processamento_texto(txt):
    nlp = spacy.load('pt_core_news_sm')
    # remover quebras de linha
    txt = txt.replace('\n',' ')
    # remover símbolos de pontuação, resultando em um array de caracteres
    txt = [char for char in txt if char not in string.punctuation]
    # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
    txt = ''.join(txt).split()
    # remover as stopwords da lista
    txt = [word for word in txt if word.lower() not in stopwords.words('portuguese')]
    
    doc = nlp(str([palavra for palavra in txt]))
    txt = [token.lemma_ for token in doc if token.pos_ == 'NOUN']
    
    return txt

In [29]:
alg ={'SVM':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
},
     'RF':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
},
     'NB':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
} }

In [30]:
for count in range(5):
    #aleatorio
    lyric_train,lyric_test,gen_train,gen_test = train_test_split(lyrics['lyric'],lyrics['gen'],test_size=0.3)
    #dados SVC
    pipeline_svc = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('svc',SVC())
    ])
    #dados NB
    pipeline_nb = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('nb',MultinomialNB())
    ])
    #dadps RF
    pipeline_rf = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('rf',RandomForestClassifier())
    ])
    
    #Etapa de Fit
    pipeline_svc.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    pipeline_nb.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    pipeline_rf.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    
    #Etapa 10-fold cross validation
    scoring = {'accuracy' : 'accuracy', 'f1_score' : 'f1_micro'}
    scores_svc = cross_validate(pipeline_svc,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    scores_nb = cross_validate(pipeline_nb,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    scores_rf = cross_validate(pipeline_rf,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    
    #Aninhamento de médias
    #SVM
    alg['SVM']['fit_time'] = np.append(alg['SVM']['fit_time'], scores_svc['fit_time'].mean())
    alg['SVM']['score_time'] = np.append(alg['SVM']['score_time'], scores_svc['score_time'].mean())
    alg['SVM']['test_accuracy'] = np.append(alg['SVM']['test_accuracy'], scores_svc['test_accuracy'].mean())
    alg['SVM']['test_f1_score'] = np.append(alg['SVM']['test_f1_score'], scores_svc['test_f1_score'].mean())
    #NB
    alg['NB']['fit_time'] = np.append(alg['NB']['fit_time'], scores_nb['fit_time'].mean())
    alg['NB']['score_time'] = np.append(alg['NB']['score_time'], scores_nb['score_time'].mean())
    alg['NB']['test_accuracy'] = np.append(alg['NB']['test_accuracy'], scores_nb['test_accuracy'].mean())
    alg['NB']['test_f1_score'] = np.append(alg['NB']['test_f1_score'], scores_nb['test_f1_score'].mean())
    #RF
    alg['RF']['fit_time'] = np.append(alg['RF']['fit_time'], scores_rf['fit_time'].mean())
    alg['RF']['score_time'] = np.append(alg['RF']['score_time'], scores_rf['score_time'].mean())
    alg['RF']['test_accuracy'] = np.append(alg['RF']['test_accuracy'], scores_rf['test_accuracy'].mean())
    alg['RF']['test_f1_score'] = np.append(alg['RF']['test_f1_score'], scores_rf['test_f1_score'].mean())

KeyboardInterrupt: 